Data Cleaning Project with Lorenzo

In [ ]:
# DATA CLEANING PIPELINE — CUSTOMER DATABASE

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load raw customer data
df = pd.read_csv('raw_customers.csv')
print(df.head())


# STEP 1: Data Quality Issues Check


def data_quality_check(df):
    quality_report = {
        'total_records': len(df),
        'duplicate_rows': df.duplicated().sum(),
        'missing_values': df.isnull().sum(),
        'invalid_emails': df[~df['email'].str.contains('@', na=False)].shape[0],
        'invalid_ages': df[(df['age'] < 0) | (df['age'] > 120)].shape[0]
    }
    return quality_report


quality_report = data_quality_check(df)
print(quality_report)


# STEP 2: Remove Duplicates


df_clean = df.copy()

initial_count = len(df_clean)

df_clean = df_clean.drop_duplicates(
    subset=['customer_id', 'email'],
    keep='first'
)

duplicates_removed = initial_count - len(df_clean)
print(f'Duplicates removed: {duplicates_removed}')


# STEP 3: Standardize Text Formatting


df_clean['first_name'] = df_clean['first_name'].str.strip()
df_clean['last_name'] = df_clean['last_name'].str.strip()
df_clean['email'] = df_clean['email'].str.strip().str.lower()
df_clean['city'] = df_clean['city'].str.strip().str.title()
df_clean['phone'] = df_clean['phone'].str.strip()

print(df_clean[['first_name', 'last_name', 'email', 'city', 'phone']])


# STEP 4: Handle Missing Values


df_clean['phone'] = df_clean['phone'].fillna('not provided')
df_clean['city'] = df_clean['city'].fillna('unknown')
df_clean['age'] = df_clean['age'].fillna(df_clean['age'].median())

missing_summary = df_clean.isnull().sum()
print(f'Remaining nulls:\n{missing_summary}')


# STEP 5: Filter Invalid Records


initial_count = len(df_clean)

df_clean = df_clean[df_clean['email'].str.contains('@', na=False)]
df_clean = df_clean[(df_clean['age'] > 0) & (df_clean['age'] < 120)]
df_clean = df_clean[df_clean['email'].notna()]

invalid_removed = initial_count - len(df_clean)
print(f'Invalid records removed: {invalid_removed}')


# STEP 6: Enrich With Calculated Fields


df_clean['full_name'] = df_clean['first_name'] + ' ' + df_clean['last_name']

df_clean['registration_date'] = pd.to_datetime(df_clean['registration_date'])

df_clean['days_since_registration'] = (
    pd.Timestamp.now() - df_clean['registration_date']
).dt.days

print(df_clean[['full_name', 'registration_date', 'days_since_registration']])


# STEP 7: Data Quality Flags


df_clean['data_quality_flag'] = np.where(
    (df_clean['phone'] == 'not provided') | (df_clean['city'] == 'unknown'),
    'Incomplete',
    'Complete'
)

quality_distribution = df_clean['data_quality_flag'].value_counts()
print(quality_distribution)


# VALIDATION SUMMARY


print('--- DATA CLEANING SUMMARY ---')
print(f'Original records: {len(df)}')
print(f'Cleaned records: {len(df_clean)}')
print(f'Records removed: {len(df) - len(df_clean)}')
print(f'Data quality distribution:\n{quality_distribution}')


# EXPORT CLEANED DATA


df_clean.to_csv('cleaned_customer.csv', index=False)
print("Cleaned data exported to 'cleaned_customer.csv'")

# Final preview
print("\n--- CLEANED DATA PREVIEW ---")
print(df_clean.head(10))
